In [31]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import shutil

def rmtree(f: Path):
    if f.is_file():
        f.unlink()
    else:
        for child in f.iterdir():
            rmtree(child)
        f.rmdir()

In [58]:
base_path = Path('data/raw_data')
files = list(base_path.glob("*/*/*.*"))
files

[PosixPath('data/raw_data/tanpa_bola/take_269663/right_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_269663/left_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_441446/right_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_441446/New Text Document.txt'),
 PosixPath('data/raw_data/tanpa_bola/take_441446/left_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_238608/right_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_238608/left_cam.jpg'),
 PosixPath('data/raw_data/bola/take_700692/right_cam.jpg'),
 PosixPath('data/raw_data/bola/take_700692/New Text Document.txt'),
 PosixPath('data/raw_data/bola/take_700692/left_cam.jpg'),
 PosixPath('data/raw_data/bola/take_086448/right_cam.jpg'),
 PosixPath('data/raw_data/bola/take_086448/New Text Document.txt'),
 PosixPath('data/raw_data/bola/take_086448/left_cam.jpg'),
 PosixPath('data/raw_data/bola/take_152294/right_cam.jpg'),
 PosixPath('data/raw_data/bola/take_152294/New Text Document.txt'),
 PosixPath('data/raw_data/bola/

In [59]:

# clean data
base_clean = base_path.parent.joinpath('cleaned')
base_clean.mkdir(parents=True, exist_ok=True)
rmtree(base_clean)


# run copy
group = []
for fpath in files:
    clazz = fpath.parent.parent.stem
    parent = fpath.parent.stem
    dst_base = base_clean.joinpath(clazz).joinpath(parent)
    dst_base.mkdir(parents=True, exist_ok=True)
    if fpath.suffix=='.txt':
        # copy note to destination directory
        dst = dst_base.joinpath('note.txt')
        shutil.copy(fpath, dst)
        
         # add [DEFAULT] to the top of the file
        with open(dst, 'r') as original: data = original.read()
        with open(dst, 'w') as modified: modified.write("[default]\n" + data)
    else:
        dst = dst_base.joinpath(fpath.name)
        shutil.copy(fpath, dst)
        
    


In [82]:
import configparser
import cv2 as cv
    
def get_note(fpath: Path):
    config = configparser.ConfigParser(default_section='note')
    config.read(str(fpath))
    return dict(config['default'])

def preprocess_image(left_path, right_path):
    left_img = cv.imread(left_path)
    left_img = cv.cvtColor(left_img, cv.COLOR_BGR2GRAY)
    
    right_img = cv.imread(right_path)
    right_img = cv.cvtColor(right_img, cv.COLOR_BGR2GRAY)
    
    img = cv.merge((left_img, right_img))
    
    return img

    
    

In [103]:
!pip install scikit-learn

6519.89s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     |████████████████████████████████| 31.2 MB 1.3 MB/s eta 0:00:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.9.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.8 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
You should consider upgrading via the '/workspace/study/code/repo/hildanida/skripsi/venv/bin/python -m pip install --upgrade pip' command.


In [117]:
# base data
from sklearn.model_selection import train_test_split

def get_cleaned_data(root):
    base_path_clean = Path(root)

    files = list(base_path_clean.glob("*/*/*.*"))
    list_dir = [fpath.parent for fpath in files]
    
    images, labels = [],[]
    for pdir in list_dir:
        lcam_path = pdir.joinpath('left_cam.jpg')
        rcam_path = pdir.joinpath('right_cam.jpg')
        note_path = pdir.joinpath('note.txt')
        if note_path.exists():
            img = preprocess_image(str(lcam_path), str(rcam_path))
            dist = float(get_note(note_path).get('z',0))
        else:
            img = preprocess_image(str(lcam_path), str(rcam_path))
            dist = float(0.0)
        images.append(img)
        labels.append(dist)
    return images, np.array(labels)
    
X,y = get_cleaned_data('data/cleaned')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [123]:
y_test

array([120., 130.,  90., 130., 130., 110., 110., 140., 150., 150., 130.,
        90.,  40., 130., 150.,  30., 130.,  40., 100.,  90., 150.,  90.,
        80.,  50.,  30.,  80.,  90.,  60.,  30.,  70.,  30., 120.,  30.,
        30.,  80.,  60., 120.])

In [135]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import Sequential

class StereoDistance(Model):
	def __init__(self, classes=1):
		super(StereoDistance, self).__init__()
		self.chan_dim=-1
		

		self.layer1 = Sequential([
			layers.Conv2D(32, (3, 3), padding="same"),
			layers.Activation("relu"),
			layers.BatchNormalization(axis=self.chan_dim),
			layers.MaxPooling2D(pool_size=(2, 2))
		])

		self.layer2 = Sequential([
			layers.Conv2D(32, (3, 3), padding="same"),
			layers.Activation("relu"),
			layers.BatchNormalization(axis=self.chan_dim),
			layers.MaxPooling2D(pool_size=(2, 2))
		])

		self.layer3 = Sequential([
			layers.Conv2D(32, (3, 3), padding="same"),
			layers.Activation("relu"),
			layers.BatchNormalization(axis=self.chan_dim),
			layers.MaxPooling2D(pool_size=(2, 2))
		])

		self.flatten = layers.Flatten()

		self.fc1 = Sequential([
			layers.Dense(512),
			layers.Activation("relu"),
			layers.BatchNormalization(),
			layers.Dropout(0.5)
		])
  
		self.fc2 = Sequential([
			layers.Dense(classes),
			layers.Activation("linear")
		])
  
  
	def call(self, x):
		x = self.layer1(x)
		x = self.layer2(x)
		x = self.layer3(x)
		x = self.flatten(x)
		x = self.fc1(x)
		x = self.fc2(x)
		return x



In [138]:
model = StereoDistance()

model.build(input_shape=(None, 480, 640, 2))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()


Model: "stereo_distance_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_80 (Sequential)  (None, 240, 320, 32)      736       
                                                                 
 sequential_81 (Sequential)  (None, 120, 160, 32)      9376      
                                                                 
 sequential_82 (Sequential)  (None, 60, 80, 32)        9376      
                                                                 
 flatten_16 (Flatten)        multiple                  0         
                                                                 
 sequential_83 (Sequential)  (None, 512)               78645760  
                                                                 
 sequential_84 (Sequential)  (None, 1)                 513       
                                                                 
Total params: 78,665,761
Trainable params: 78,66

In [139]:

history = model.fit(X_train, 
                   y_train, 
                   epochs=20, 
                   batch_size=512,
                   validation_data=(X_test, y_test))

ValueError: Data cardinality is ambiguous:
  x sizes: 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 4